In [54]:
!pip install xgboost

In [56]:
#import os
# spark_version = 'spark-3.4.0'
#spark_version = 'spark-3.4.0'
#os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
#!apt-get update
#!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
#!tar xf $SPARK_VERSION-bin-hadoop3.tgz
#!pip install -q findspark

# Set Environment Variables
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
#os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
#import findspark
#findspark.init()

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#models
from xgboost import XGBRegressor

#  Import and read the charity_data.csv.
import pandas as pd

# Get data from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Prepare Data

In [30]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/energy_weather_joined.csv')
df.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
1,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2,2015-01-01 05:00:00+01:00,410.0,178.0,0.0,4038.0,3590.0,156.0,0.0,0.0,0.0,...,2,321,0.0,0.0,0.0,0,800,clear,sky is clear,01n
3,2015-01-01 07:00:00+01:00,408.0,172.0,0.0,4030.0,3208.0,160.0,0.0,0.0,0.0,...,2,321,0.0,0.0,0.0,0,800,clear,sky is clear,01n
4,2015-01-01 11:00:00+01:00,421.0,226.0,0.0,3931.0,3845.0,166.0,0.0,0.0,0.0,...,1,255,0.0,0.0,0.0,0,800,clear,sky is clear,01d


In [31]:
# get the columns name as reference for below
df.columns

Index(['time', 'generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil coal-derived gas', 'generation fossil gas',
       'generation fossil hard coal', 'generation fossil oil',
       'generation fossil oil shale', 'generation fossil peat',
       'generation geothermal', 'generation hydro pumped storage aggregated',
       'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation marine',
       'generation nuclear', 'generation other', 'generation other renewable',
       'generation solar', 'generation waste', 'generation wind offshore',
       'generation wind onshore', 'forecast solar day ahead',
       'forecast wind offshore eday ahead', 'forecast wind onshore day ahead',
       'total load forecast', 'total load actual', 'price day ahead',
       'price actual', 'city_name', 'temp', 'temp_min', 'temp_max', 'pressure',
       'humidity', 'wind_speed',

In [32]:
df=df.drop(["generation marine", "generation wind offshore", "forecast solar day ahead", "forecast wind onshore day ahead", "forecast wind onshore day ahead", "forecast wind offshore eday ahead", "total load forecast", "price day ahead", "generation hydro pumped storage aggregated"], axis=1)

In [33]:
df.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
1,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2,2015-01-01 05:00:00+01:00,410.0,178.0,0.0,4038.0,3590.0,156.0,0.0,0.0,0.0,...,2,321,0.0,0.0,0.0,0,800,clear,sky is clear,01n
3,2015-01-01 07:00:00+01:00,408.0,172.0,0.0,4030.0,3208.0,160.0,0.0,0.0,0.0,...,2,321,0.0,0.0,0.0,0,800,clear,sky is clear,01n
4,2015-01-01 11:00:00+01:00,421.0,226.0,0.0,3931.0,3845.0,166.0,0.0,0.0,0.0,...,1,255,0.0,0.0,0.0,0,800,clear,sky is clear,01d


In [34]:
df=df.drop(["weather_id", "weather_icon", "weather_description", "weather_main", "temp_min", "city_name", "temp_max"], axis=1)
df.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,price actual,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,65.41,270.475,1001,77,1,62,0.0,0.0,0.0,0
1,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,56.04,269.686,1002,78,0,23,0.0,0.0,0.0,0
2,2015-01-01 05:00:00+01:00,410.0,178.0,0.0,4038.0,3590.0,156.0,0.0,0.0,0.0,...,53.63,270.292,1004,71,2,321,0.0,0.0,0.0,0
3,2015-01-01 07:00:00+01:00,408.0,172.0,0.0,4030.0,3208.0,160.0,0.0,0.0,0.0,...,51.43,270.292,1004,71,2,321,0.0,0.0,0.0,0
4,2015-01-01 11:00:00+01:00,421.0,226.0,0.0,3931.0,3845.0,166.0,0.0,0.0,0.0,...,59.86,284.824,1006,55,1,255,0.0,0.0,0.0,0


In [35]:
#split time (datetime format) column into date and time
datetimezone = df['time'].astype("string")
datelist = []
timelist = []

for datetimerow in datetimezone:
    date, time = datetimerow.split(" ", 1)
    time, timezone = time.split("+", 1)
    datelist.append(date)
    timelist.append(time)

In [36]:
monthlist = []
daylist = []

for date in datelist:
  year, yearless = date.split("-", 1)
  month, day = yearless.split("-", 1)
  daylist.append(day)
  monthlist.append(month)

df['day'] = daylist
df['month'] = monthlist

In [37]:
hourlist = []

for time in timelist:
  hour, hourless = time.split(":", 1)
  hourlist.append(hour)

df['hour'] = hourlist

In [38]:
df=df.drop(["time"], axis=1)

In [39]:
df['month'] = df['month'].astype('int')
df['day'] = df['day'].astype('int')
df['hour'] = df['hour'].astype('int')

# Machine Learning Regression model

In [82]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(df.loc[:,df.columns != "price actual"])

In [83]:
# Split our preprocessed data into our features and target arrays
y = df["price actual"].values
#X = application_df.drop(["IS_SUCCESSFUL"],axis=1).values
X = dummies
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [84]:
model = XGBRegressor()

In [85]:
X = df.loc[:,df.columns != "price actual"]
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [86]:
score = model.score(X_train, y_train)
print(f"score: {score}")

score: 0.9402385649066317


In [81]:
pred_price = model.predict(X_test)
# summarize prediction
#average = sum(prediction)/len(prediction)

#predicted price
print('Predicted price: %.3f euros per megawatts-hour' % pred_price[0])

Predicted price: 43.663 euros per megawatts-hour


# Analyze

In [91]:
analyze_df = X_test.copy()
analyze_df["predicted Price"] = pred_price

In [98]:
analyze_df['y_test'] = y_test

In [99]:
analyze_df.head()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage consumption,...,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,day,month,hour,predicted Price,y_test
114972,366.0,862.0,0.0,3737.0,4287.0,220.0,0.0,0.0,0.0,2679.0,...,90,0.0,0.0,0.0,90,27,11,3,43.663120,49.93
137564,402.0,558.0,0.0,4967.0,2093.0,314.0,0.0,0.0,0.0,1046.0,...,340,0.0,0.0,0.0,20,9,11,18,68.884216,70.31
95107,320.0,420.0,0.0,4375.0,4812.0,254.0,0.0,0.0,0.0,565.0,...,21,0.0,0.0,0.0,20,7,6,3,52.124657,50.44
148802,334.0,0.0,0.0,5098.0,1320.0,277.0,0.0,0.0,0.0,0.0,...,330,0.0,0.0,0.0,20,13,5,11,37.767128,35.06
164282,444.0,957.0,0.0,4689.0,4889.0,390.0,0.0,0.0,0.0,0.0,...,80,0.0,0.0,0.0,40,5,11,9,62.576092,63.68


In [110]:
hour_price_pred = analyze_df.groupby('hour')['predicted Price'].mean()
hour_price_pred

hour
0     55.314877
1     51.837425
2     49.453510
3     47.845913
4     47.765892
5     48.668343
6     51.843586
7     56.000748
8     60.121056
9     61.957214
10    62.578320
11    62.153675
12    61.662483
13    61.017929
14    59.541939
15    57.689182
16    57.783165
17    58.318871
18    62.195004
19    64.500862
20    65.538124
21    65.055107
22    62.285549
23    58.638683
Name: predicted Price, dtype: float32

In [111]:
hour_price_act = analyze_df.groupby('hour')['y_test'].mean()
hour_price_act

hour
0     55.390415
1     51.931927
2     49.561792
3     47.819473
4     47.336659
5     48.589482
6     52.144768
7     55.894864
8     60.163665
9     62.022118
10    62.647173
11    62.288204
12    61.784646
13    61.054436
14    59.513694
15    57.591996
16    57.623458
17    58.319859
18    62.097346
19    64.555207
20    65.614381
21    65.422882
22    62.205323
23    58.433864
Name: y_test, dtype: float64

In [112]:
hour_price_act - hour_price_pred

hour
0     0.075539
1     0.094502
2     0.108282
3    -0.026440
4    -0.429233
5    -0.078860
6     0.301182
7    -0.105884
8     0.042610
9     0.064904
10    0.068853
11    0.134529
12    0.122163
13    0.036506
14   -0.028245
15   -0.097187
16   -0.159707
17    0.000989
18   -0.097657
19    0.054345
20    0.076257
21    0.367775
22   -0.080226
23   -0.204819
dtype: float64